<a href="https://colab.research.google.com/github/nunsongi/quito-risk-mapping-python/blob/main/Limpieza_Sismos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Limpieza y Preparación de Datos Sísmicos de Quito 🇪🇨*
Este notebook tiene como objetivo cargar los catálogos sísmicos completos, filtrarlos para incluir solo los eventos dentro de la ciudad de Quito y guardar los resultados como un archivo CSV limpio para su posterior análisis.

## Contenido

Importación de librerías

Carga Inicial del Archivo TXT

Filtrado Geográfico (Quito)

Revisión Final y Exportación a CSV

## Importación de librerías

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

## Carga Inicial del Archivo TXT
Los archivos que tienen datos sobre los sismos se encuentran en formato .txt lo que no es adecuado para el analisis. Se debe convertir a CSV

In [ ]:
# Carga y Extracción de Datos

# Magnitud de los Sismos
columns_mag = [
    'event', 'orig_id', 'magnitude', 'magnitude_uncertainty',
    'magnitude_type', 'methodID', 'stationCount', 'agencyID'
]

archive_txt_mag = pd.read_csv(r'/content/cat_magnitud_2012-jul2025.txt',
                              sep='\,',
                              skiprows=27,
                              names=columns_mag,
                              encoding='utf-8')

print("Magnitud de los Sismos")
display(archive_txt_mag.head())

## Filtrado Geográfico (Quito)

## Revisión Final y Exportación a CSV